# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Not for Grading

### Setup Steps:

In [0]:
#@title Please enter your registration id to start: (e.g. P181900101) { run: "auto", display-mode: "form" }
Id = "P19A07E_test" #@param {type:"string"}


In [0]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "981234568" #@param {type:"string"}


In [0]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()
  
notebook="M2W7_CS_3_Rocchio_feedback_search_updated" #name of the notebook
Answer = "Ungraded"
def setup():
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/CaseStudies/feedback_search.zip")  
    ipython.magic("sx unzip feedback_search.zip")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      print("Your submission is successful.")
      print("Ref Id:", submission_id)
      print("Date of submission: ", r["date"])
      print("Time of submission: ", r["time"])
      print("View your submissions: https://iiith-aiml.talentsprint.com/notebook_submissions")
      print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if Additional: return Additional      
    else: raise NameError('')
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getAnswer():
  try:
    return Answer
  except NameError:
    print ("Please answer Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
    from IPython.display import HTML
    HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id))
  
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


In [0]:
%cd feedback_search/

/content/feedback_search


##Implementing Rocchio feedback  search


In [0]:
"""
Implementation of an information retrieval system that exploits user-provided
relevance feedback to improve the search results returned by Google.
"""
import sys
import logging
import threading
import os
import mock_feedback, mock_query_and_scraping
import query as query_file
import feedback
import enhance_query
import index
import scrape
import preprocess
import constants

logger = logging.getLogger('feedback_search')
logger.propagate = False # do not log in console
os.makedirs('logs', exist_ok=True) # create directory for logs if it's not there already
handler = logging.FileHandler('logs/feedback_search.log')
formatter = logging.Formatter(
    fmt='[%(asctime)s %(levelname)s]\t%(message)s',
    datefmt='%d-%m-%Y %H:%M:%S')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)


def main(query, precision=0.8, is_test=False):
    """
    Main routine, 
    Takes initial query and target_precision provided as input,
    Until target_precision is achieved:
        Runs enhanced query, asks user's feedback, computes new precision.
    """

    try:
        target_precision = float(precision)
    except ValueError:
        print('<precision> must be a float between 0 and 1 !')
        return

    if target_precision > 1 or target_precision <0:
        print('<precision> must be a float between 0 and 1 !')
        return

    logger.info('\n\n ========================================================================\n\n')
    logger.info('[MAIN]\t\t Started with args: QUERY = %s, PRECISION = %s', query, target_precision)

#import pdb; pdb.set_trace()
    achieved_precision = 0

    # Build one index for each zone of the documents (see enhance_query):
    indexers = {zone: index.UnigramIndexer(zone) for zone in ['title', 'summary', 'content']}
    bigram_indexers = {zone: index.BigramIndexer(zone) for zone in ['title', 'summary', 'content']}

    query_optimizer = enhance_query.RocchioQueryOptimizer()

    while (achieved_precision < target_precision):
        logger.info('[MAIN]\t\t achieved precision = %s vs target precision = %s, optimizing...', achieved_precision, target_precision)
        print('Parameters:')
        print('Query = {}'.format(query))
        print('Precision = {}'.format(target_precision))
        print('')
        
        if not is_test:
            results = query_file.query_google(query)

            # Fetch the whole documents by scraping the urls in results, as a background task
            scraping_thread = threading.Thread(target=scrape.add_url_content, args=(query, results))
            scraping_thread.start()

            if len(results) < 10:
                print('Too few results, aborting...')
                break

            # Ask feedback to user, store feedback in results dict directly
            feedback.ask_feedback(results)
            scraping_thread.join() # make sure all the documents have been scraped

        elif is_test:
            results = mock_query_and_scraping.load_query_results(query)
            mock_feedback.mock_feedback(results, query=query)

        relevant = [doc['id'] for doc in results if doc['relevant']]
        non_relevant = [doc['id'] for doc in results if not doc['relevant']]
        achieved_precision = len(relevant)/len(results) if results else 0

        if achieved_precision == 0:
            print('Precision@10 is 0, aborting...')
            break

        logger.info('[MAIN]\t\t orginal query: %s', query)
        query = preprocess.split_remove_punctuation(query)
        if constants.USE_STEMMING:
            query = preprocess.stem(query)
        logger.info('[MAIN]\t\t preprocessed query: %s', query)

        indexing_threads = []
        for zone in indexers:
            indexers[zone].reset()
            t = threading.Thread(target=indexers[zone].index, args=(results, query))
            t.start()
            indexing_threads.append(t)
        
        for zone in bigram_indexers:
            bigram_indexers[zone].reset()
            t = threading.Thread(target=bigram_indexers[zone].index, args=(results, query))
            t.start()
            indexing_threads.append(t)

        for t in indexing_threads:
            t.join()

        print('Achieved precision: ', achieved_precision)

        query = query_optimizer.enhance(query, indexers, relevant, non_relevant, bigram_indexers=bigram_indexers)

        if is_test:
            # in tests we do unly one run of the query optimizer with mock feedback and analyze it
            return query

if __name__ == '__main__':
    main('cricket', precision=0.9)

### Please answer the questions below to complete the experiment:




In [0]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good, But Not Challenging for me" #@param ["Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [0]:
#@title If it was very easy, what more you would have liked to have been added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "test'\"" #@param {type:"string"}

In [0]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["Yes", "No"]

In [0]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")